In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

In [18]:
data = pd.read_csv('/mnt/New_Volume/Work_From_Home/Data/spam.csv')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [19]:
data.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [20]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1,inplace= True)

In [21]:
data.columns = ['label','message']

In [22]:
data['label'].value_counts()

ham     5854
spam     922
Name: label, dtype: int64

# Step 1 convert label to numbers

In [23]:
data['label'].replace({'ham':1, 'spam':0},inplace= True)

In [24]:
stop_words = stopwords.words('english')

# Step 2 : Covert to lower/upper case the complete data set. Remove punctuations and stop words

In [25]:
data['message'] = data['message'].str.lower()

In [26]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [27]:
def text_processing(mess):
    no_punc = [char for char in mess if char not in string.punctuation]      # removing punctuation
    no_punc = ''.join(no_punc)
    final = [word for word in no_punc.split() if word not in stop_words]    # removing stop words
    return final

In [28]:
# data['message'] = data['message'].apply(text_processing)

In [29]:
data['message'].iloc[0:3]

0    go until jurong point, crazy.. available only ...
1                        ok lar... joking wif u oni...
2    free entry in 2 a wkly comp to win fa cup fina...
Name: message, dtype: object

# Step 3 Create Term Document Matrix or Document Term Matrix

In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# TfidfVextorizefr :-> creates a matrix of words and instead of ones we have importance values in matrix

In [31]:
# bow_transformer = CountVectorizer(analyzer=text_processing).fit(data['message'])
# bow_transformer.vocabulary_

In [32]:
vectorizer = TfidfVectorizer(stop_words='english')
x = vectorizer.fit_transform(data['message'])

In [35]:
z = x.toarray()
z

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
np.unique(z[0])

array([0.        , 0.16973059, 0.19590565, 0.20598878, 0.24061153,
       0.2609886 , 0.2802746 , 0.28258599, 0.29043733, 0.3040437 ,
       0.3083043 , 0.31846703, 0.34109695])

# Now i have a matrix of zeros and ones we can feed it to any algorithm as we have targets as well

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
y = data['label']

In [44]:
x_train,x_test, y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

In [46]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()

In [47]:
log_reg.fit(x_train,y_train)

LogisticRegression()

In [48]:
from sklearn.metrics import accuracy_score,classification_report

In [50]:
y_pred = log_reg.predict(x_test)
log_reg_score = accuracy_score(y_test,y_pred)
log_reg_score

0.9609144542772862

In [51]:
classification_report(y_test,y_pred, output_dict=True)

{'0': {'precision': 0.9925373134328358,
  'recall': 0.7189189189189189,
  'f1-score': 0.8338557993730407,
  'support': 185},
 '1': {'precision': 0.9574468085106383,
  'recall': 0.9991460290350128,
  'f1-score': 0.9778520685332219,
  'support': 1171},
 'accuracy': 0.9609144542772862,
 'macro avg': {'precision': 0.9749920609717371,
  'recall': 0.8590324739769659,
  'f1-score': 0.9058539339531313,
  'support': 1356},
 'weighted avg': {'precision': 0.962234229904891,
  'recall': 0.9609144542772862,
  'f1-score': 0.9582065598351146,
  'support': 1356}}

# NLP WITH Neural Networks